In [95]:
import pandas as pd
import re
from datetime import datetime, timedelta
import datetime as dt

In [58]:
earthquakes=pd.read_csv('./VillarricaEarthquakes.csv')

In [59]:
eruptions=pd.read_csv('./VillarricaEruptions.csv')

# Exploring datasets

In [60]:
eruptions.head()

,volcano_number,volcano_name,eruption_number,eruption_category,vei,start_year,start_month,start_day,latitude,longitude,date
0,357120,Villarrica,12184,Confirmed Eruption,1.0,2009.0,11.0,22.0,-39.42,-71.93,2009-11-22
1,357120,Villarrica,12183,Confirmed Eruption,1.0,2009.0,1.0,29.0,-39.42,-71.93,2009-01-29
2,357120,Villarrica,12182,Confirmed Eruption,1.0,2008.0,10.0,26.0,-39.42,-71.93,2008-10-26
3,357120,Villarrica,12181,Confirmed Eruption,1.0,2004.0,8.0,5.0,-39.42,-71.93,2004-08-05
4,357120,Villarrica,12180,Confirmed Eruption,1.0,2003.0,5.0,23.0,-39.42,-71.93,2003-05-23


In [61]:
eruptions=eruptions.rename(columns={'latitude':'volcano_latitude','longitude': 'volcano_longitude'})

In [62]:
eruptions['date']=pd.to_datetime(eruptions['date'],format="%Y/%m/%d")

In [63]:
eruptions['Days since last eruption']=eruptions['date'].diff(periods=-1).dt.days

In [64]:
eruptions.loc[eruptions['eruption_number']==12150,'Days since last eruption']=700

In [65]:
earthquakes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1726 entries, 0 to 1725
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         1726 non-null   object 
 1   mag        1726 non-null   float64
 2   longitude  1726 non-null   float64
 3   latitude   1726 non-null   float64
 4   date       1726 non-null   object 
dtypes: float64(3), object(2)
memory usage: 67.5+ KB


In [66]:
earthquakes['date']=pd.to_datetime(earthquakes['date'],format="%Y/%m/%d")

# Merging datasets

In [67]:
for day in range(0,31):    
    earthquakes[f'date-{day}']=earthquakes['date']-timedelta(days=day)

In [68]:
for x in range(0, 31):
    globals()['villarrica%s' % x] = earthquakes.join(eruptions.set_index('date'), on=f'date-{x}')
    globals()['villarrica%s' % x] = globals()['villarrica%s' % x].dropna()

In [69]:
villarrica=pd.DataFrame()
for i in range(0,31):
    villarrica=pd.concat([villarrica, globals()['villarrica%s' % i]])

In [70]:
villarrica0=earthquakes.join(eruptions.set_index('date'), on='date-0')

In [71]:
villarrica=pd.concat([villarrica,villarrica0])

In [72]:
villarrica=villarrica.drop_duplicates()

# Final adjustments

In [73]:
villarrica

,id,mag,longitude,latitude,date,date-0,date-1,date-2,date-3,date-4,...,volcano_name,eruption_number,eruption_category,vei,start_year,start_month,start_day,volcano_latitude,volcano_longitude,Days since last eruption
747,usp000h4b2,5.9,-74.967,-39.634,2009-11-22,2009-11-22,2009-11-21,2009-11-20,2009-11-19,2009-11-18,...,Villarrica,12184.0,Confirmed Eruption,1.0,2009.0,11.0,22.0,-39.42,-71.93,297.0
1042,usp000d1hj,3.6,-74.294,-38.658,2004-08-05,2004-08-05,2004-08-04,2004-08-03,2004-08-02,2004-08-01,...,Villarrica,12181.0,Confirmed Eruption,1.0,2004.0,8.0,5.0,-39.42,-71.93,440.0
1041,usp000d1mc,2.9,-71.392,-36.886,2004-08-06,2004-08-06,2004-08-05,2004-08-04,2004-08-03,2004-08-02,...,Villarrica,12181.0,Confirmed Eruption,1.0,2004.0,8.0,5.0,-39.42,-71.93,440.0
1611,usp0000mcq,4.8,-73.506,-38.184,1977-01-28,1977-01-28,1977-01-27,1977-01-26,1977-01-25,1977-01-24,...,Villarrica,12166.0,Confirmed Eruption,1.0,1977.0,1.0,26.0,-39.42,-71.93,1916.0
1699,iscgem879719,5.8,-73.555,-38.534,1960-07-05,1960-07-05,1960-07-04,1960-07-03,1960-07-02,1960-07-01,...,Villarrica,12161.0,Uncertain Eruption,1.0,1960.0,7.0,2.0,-39.42,-71.93,604.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1721,iscgem893991,6.2,-73.519,-38.589,1951-01-03,1951-01-03,1951-01-02,1951-01-01,1950-12-31,1950-12-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1722,iscgem896480,6.9,-72.676,-37.665,1949-04-20,1949-04-20,1949-04-19,1949-04-18,1949-04-17,1949-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1723,iscgem901645,7.0,-73.630,-42.041,1940-10-11,1940-10-11,1940-10-10,1940-10-09,1940-10-08,1940-10-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1724,iscgem901785,7.8,-72.315,-36.305,1939-01-25,1939-01-25,1939-01-24,1939-01-23,1939-01-22,1939-01-21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
villarrica['was_erupted']=villarrica['volcano_number'].fillna(0)

In [75]:
villarrica['was_erupted'].loc[villarrica['was_erupted']==357120]=1

C:\Users\Pedro\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [76]:
villarrica=villarrica.sort_values('date')

In [77]:
villarrica['Days since last eruption']=villarrica['date'].diff(periods=-1).dt.days

In [78]:
villarrica['Days since last eruption']

1725   -1791.0
1724    -625.0
1723   -3113.0
1722    -623.0
1721    -854.0
         ...  
4         -3.0
3         -3.0
2         -2.0
1         -3.0
0          NaN
Name: Days since last eruption, Length: 1760, dtype: float64

In [79]:
villarrica.shape

(1760, 48)

In [80]:
i=0
last_eruption_date=datetime.strptime('1927/01/01', '%Y/%m/%d')
while i<villarrica.shape[0]:
    if villarrica.iloc[i,47]==1:
        villarrica.iloc[i,46]=(villarrica.iloc[i,4]-last_eruption_date)
        last_eruption_date=villarrica.iloc[i,4]
    else:
        villarrica.iloc[i,46]=(villarrica.iloc[i,4]-last_eruption_date)
    i+=1
print(last_eruption_date)

2009-11-22 00:00:00


In [109]:
villarrica['Days since last eruption']=villarrica['Days since last eruption'].apply(lambda x: re.findall('\d*',str(x))[0]).astype(int)

# Exporting merged dataset